<a href="https://colab.research.google.com/github/MusaibSyd/Transfer-Learning/blob/main/Transfer_Learning_Resent_V2_50_Food_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import zipfile
import os

In [9]:
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [15]:
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/gr

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir  = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

print ("Training Images:")

train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")

print ("Test Images:")

test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")

Training Images:
Found 750 images belonging to 10 classes.
Test Images:
Found 2500 images belonging to 10 classes.


In [17]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

In [19]:
import tensorflow as tf
import tensorflow_hub as hub 
from tensorflow.keras import layers

In [27]:
def create_model (model_url, num_classes=10):
  #Download the pretrained model and asave it as a keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=(224,224,3))
  #Create our model

  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation='Softmax', name='output_layer')
  ])
  return model
  

In [28]:
#creating and testing ResNet Model

resnet_model = create_model(resnet_url)

In [33]:
resnet_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [32]:
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [35]:
resnet_model.fit(train_data_10_percent,
                 epochs=5,
                 validation_data=test_data_10_percent)

Epoch 1/5
24/24 [==============================] - 458s 20s/step - loss: 2.1054 - accuracy: 0.2827 - val_loss: 1.3283 - val_accuracy: 0.5652
Epoch 2/5
24/24 [==============================] - 455s 20s/step - loss: 0.9777 - accuracy: 0.6973 - val_loss: 0.9022 - val_accuracy: 0.7196
Epoch 3/5
24/24 [==============================] - 453s 19s/step - loss: 0.6581 - accuracy: 0.8080 - val_loss: 0.7784 - val_accuracy: 0.7548
Epoch 4/5
24/24 [==============================] - 453s 19s/step - loss: 0.4899 - accuracy: 0.8667 - val_loss: 0.7227 - val_accuracy: 0.7672
Epoch 5/5
24/24 [==============================] - 450s 19s/step - loss: 0.3922 - accuracy: 0.9027 - val_loss: 0.6816 - val_accuracy: 0.7776


In [36]:
import matplotlib.pyplot as plt


In [37]:
def plot_loss_curves(history):
  
  loss = history.history['loss']
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epcohs = range(len(history.history["loss"]))

  #plot loss

  plt.plot(epoch, loss, label="training_loss")
  plt.plot(epoch, val_loss, label="val_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  #accuracy
  plt.figure()
  plt.plot(epoch, accuracy, label="training_accuracy")
  plt.plot(epoch, val_accuracy, label="val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend()